# VQE框架性能基准测试 - 研究指南

## 概述

本指南面向研究人员，介绍如何使用 `vqe_bench_new.py` 进行量子计算框架的性能比较研究。假设读者已有量子计算和变分量子算法(VQE)的基础知识。

### 目标读者
- 量子计算研究人员
- 需要评估不同量子计算框架性能的研究者
- 希望进行量子算法性能比较的学者

### 指南结构
1. 框架架构概览
2. 快速开始
3. 配置系统详解
4. 基准测试执行
5. 结果分析与可视化
6. 高级应用与自定义
7. 研究案例
8. 最佳实践与建议

## 1. 框架架构概览

### 1.1 分层配置系统设计

`vqe_bench_new.py` 采用分层配置系统，将配置分为两个层次：

- **核心用户层**：最常用且易于理解的参数
- **高级研究层**：专家级设置，用于深入、特定的基准测试

In [ ]:
# 导入配置系统
from vqe_config import merge_configs, get_quick_start_config

# 获取默认配置
config = merge_configs()

# 获取快速开始配置
quick_config = get_quick_start_config()

print("默认配置:")
for key, value in config.items():
    if not isinstance(value, dict):
        print(f"  {key}: {value}")

### 1.2 核心组件架构

框架采用面向对象设计，主要包含以下核心组件：

1. **FrameworkWrapper**：抽象基类，为不同量子框架提供统一接口
2. **VQERunner**：VQE执行引擎，封装单次VQE运行的完整逻辑
3. **BenchmarkController**：控制器，协调整个测试流程
4. **VQEBenchmarkVisualizer**：可视化器，生成性能分析图表

In [ ]:
# 展示框架核心组件关系
from vqe_bench_new import BenchmarkController, VQEBenchmarkVisualizer

# 简要展示架构流程
config = merge_configs()
controller = BenchmarkController(config)
print("BenchmarkController 已创建")
print("框架核心组件:")
print("  - BenchmarkController: 协调整个测试流程")
print("  - FrameworkWrapper: 为不同框架提供统一接口")
print("  - VQERunner: 执行单次VQE运行")
print("  - VQEBenchmarkVisualizer: 生成可视化图表")

### 1.3 支持的量子计算框架

框架目前支持三个主流量子计算框架：

- **Qiskit**：IBM开发的量子计算框架
- **PennyLane**：Xanadu开发的量子机器学习框架
- **Qibo**：轻量级量子计算框架，支持多种后端

In [ ]:
# 检查可用框架
try:
    import qiskit
    qiskit_available = True
except ImportError:
    qiskit_available = False

try:
    import pennylane
    pennylane_available = True
except ImportError:
    pennylane_available = False

try:
    import qibo
    qibo_available = True
except ImportError:
    qibo_available = False

print("框架可用性检查:")
print(f"  Qiskit: {'✓ 可用' if qiskit_available else '✗ 不可用'}")
print(f"  PennyLane: {'✓ 可用' if pennylane_available else '✗ 不可用'}")
print(f"  Qibo: {'✓ 可用' if qibo_available else '✗ 不可用'}")

# 根据可用性调整框架列表
available_frameworks = []
if qiskit_available:
    available_frameworks.append("Qiskit")
if pennylane_available:
    available_frameworks.append("PennyLane")
if qibo_available:
    available_frameworks.append("Qibo")

print(f"\n可用框架: {available_frameworks}")

## 2. 快速开始

### 2.1 环境准备

确保已安装必要的依赖：

In [ ]:
# 检查依赖
import sys

def check_package(package_name):
    try:
        __import__(package_name)
        return True
    except ImportError:
        return False

required_packages = {
    'numpy': 'numpy',
    'matplotlib': 'matplotlib',
    'scipy': 'scipy',
    'psutil': 'psutil',
    'qibo': 'qibo',
    'qiskit': 'qiskit',
    'pennylane': 'pennylane'
}

print("依赖检查:")
all_available = True
for display_name, package_name in required_packages.items():
    available = check_package(package_name)
    status = "✓" if available else "✗"
    print(f"  {status} {display_name}")
    if not available:
        all_available = False

if not all_available:
    print("\n缺少依赖。请运行以下命令安装:")
    missing_packages = [pkg for pkg in required_packages.values() if not check_package(pkg)]
    print(f"pip install {' '.join(missing_packages)}")
else:
    print("\n所有依赖已满足，可以继续。")

### 2.2 最小可行配置和运行

In [ ]:
# 快速开始示例
from vqe_config import get_quick_start_config
from vqe_bench_new import BenchmarkController, VQEBenchmarkVisualizer

# 获取快速配置
config = get_quick_start_config()
print("快速配置:")
for key, value in config.items():
    if not isinstance(value, dict):
        print(f"  {key}: {value}")

# 根据可用框架调整配置
if len(available_frameworks) > 0:
    config["frameworks_to_test"] = [available_frameworks[0]]  # 只使用第一个可用框架
    print(f"\n调整后框架: {config['frameworks_to_test']}")

In [ ]:
# 运行基准测试
if len(available_frameworks) > 0:
    controller = BenchmarkController(config)
    print("\n开始运行基准测试...")
    results = controller.run_all_benchmarks()
    print("基准测试完成!")
else:
    print("没有可用的框架，无法运行基准测试。")
    results = None

In [ ]:
# 生成可视化
if results:
    visualizer = VQEBenchmarkVisualizer(results, config)
    visualizer.plot_dashboard()
    print("可视化仪表盘已生成")
else:
    print("无法生成可视化，因为没有基准测试结果。")

## 3. 配置系统详解

### 3.1 核心配置参数

#### 3.1.1 基本问题设置

In [ ]:
# 自定义基本配置
custom_config = {
    # 量子比特数范围
    "n_qubits_range": [4, 6, 8, 10],
    
    # 要测试的框架
    "frameworks_to_test": available_frameworks if len(available_frameworks) > 0 else ["Qiskit"],
    
    # Ansatz类型
    "ansatz_type": "HardwareEfficient",  # 或 "QAOA"
    
    # 优化器选择
    "optimizer": "COBYLA",  # 或 "SPSA", "L-BFGS-B"
    
    # 运行次数(统计可靠性)
    "n_runs": 5,
    
    # 实验名称
    "experiment_name": "My_Research_Benchmark"
}

print("自定义基本配置:")
for key, value in custom_config.items():
    print(f"  {key}: {value}")

#### 3.1.2 Ansatz配置

In [ ]:
# Ansatz详细配置
ansatz_config = {
    "ansatz_details": {
        # Ansatz层数
        "n_layers": 4,
        
        # 纠缠模式
        "entanglement_style": "linear"  # "linear", "circular", "full"
    }
}

print("Ansatz配置:")
for key, value in ansatz_config["ansatz_details"].items():
    print(f"  {key}: {value}")

### 3.2 高级研究参数

#### 3.2.1 物理问题定义

In [ ]:
# 物理模型配置
problem_config = {
    "problem": {
        # 模型类型
        "model_type": "TFIM_1D",  # 目前主要支持一维横向场伊辛模型
        
        # 边界条件
        "boundary_conditions": "periodic",  # "periodic" 或 "open"
        
        # 物理参数
        "j_coupling": 1.0,      # 相互作用强度
        "h_field": 1.0,         # 横向场强度
        "disorder_strength": 0.0  # 无序强度
    }
}

print("物理问题配置:")
for key, value in problem_config["problem"].items():
    print(f"  {key}: {value}")

#### 3.2.2 优化器配置

In [ ]:
# 优化器详细配置
optimizer_config = {
    "optimizer_details": {
        # 优化器特定参数
        "options": {
            "COBYLA": {"tol": 1e-5, "rhobeg": 1.0},
            "SPSA": {"learning_rate": 0.05, "perturbation": 0.05},
            "L-BFGS-B": {"ftol": 1e-7, "gtol": 1e-5}
        },
        
        # 最大评估次数
        "max_evaluations": 500,
        
        # 收敛阈值
        "accuracy_threshold": 1e-4
    }
}

print("优化器配置:")
print("  选项:")
for optimizer, params in optimizer_config["optimizer_details"]["options"].items():
    print(f"    {optimizer}: {params}")
print(f"  最大评估次数: {optimizer_config['optimizer_details']['max_evaluations']}")
print(f"  收敛阈值: {optimizer_config['optimizer_details']['accuracy_threshold']}")

### 3.3 配置合并与验证

In [ ]:
# 合并配置
from vqe_config import merge_configs, validate_config

# 合并多个配置
merged_config = merge_configs(custom_config, ansatz_config, problem_config, optimizer_config)

print("合并后的配置:")
for key, value in merged_config.items():
    if not isinstance(value, dict):
        print(f"  {key}: {value}")
    elif key in ["problem", "ansatz_details", "optimizer_details"]:
        print(f"  {key}:")
        for subkey, subvalue in value.items():
            print(f"    {subkey}: {subvalue}")

In [ ]:
# 验证配置
is_valid, errors = validate_config(merged_config)
if not is_valid:
    print("配置错误:")
    for error in errors:
        print(f"  - {error}")
else:
    print("配置验证通过")

## 4. 基准测试执行

### 4.1 单框架深度测试

In [ ]:
# 单框架测试配置
if len(available_frameworks) > 0:
    single_framework_config = merge_configs({
        "frameworks_to_test": [available_frameworks[0]],  # 只使用第一个可用框架
        "n_qubits_range": [4, 6, 8, 10, 12],
        "n_runs": 3,  # 减少运行次数以节省时间
        "experiment_name": "Single_Framework_Deep_Analysis"
    })
    
    print("单框架测试配置:")
    print(f"  框架: {single_framework_config['frameworks_to_test']}")
    print(f"  量子比特数: {single_framework_config['n_qubits_range']}")
    print(f"  运行次数: {single_framework_config['n_runs']}")
    
    # 执行测试
    controller = BenchmarkController(single_framework_config)
    print("\n开始单框架深度测试...")
    single_results = controller.run_all_benchmarks()
    print("单框架测试完成!")
else:
    print("没有可用的框架，无法运行单框架测试。")
    single_results = None

### 4.2 多框架比较

In [ ]:
# 多框架比较配置
if len(available_frameworks) >= 2:
    multi_framework_config = merge_configs({
        "frameworks_to_test": available_frameworks[:2],  # 使用前两个可用框架
        "n_qubits_range": [4, 6, 8],
        "n_runs": 3,  # 减少运行次数以节省时间
        "experiment_name": "Multi_Framework_Comparison"
    })
    
    print("多框架比较配置:")
    print(f"  框架: {multi_framework_config['frameworks_to_test']}")
    print(f"  量子比特数: {multi_framework_config['n_qubits_range']}")
    print(f"  运行次数: {multi_framework_config['n_runs']}")
    
    # 执行测试
    controller = BenchmarkController(multi_framework_config)
    print("\n开始多框架比较测试...")
    multi_results = controller.run_all_benchmarks()
    print("多框架比较测试完成!")
else:
    print("可用框架不足，无法运行多框架比较。")
    multi_results = None

## 5. 结果分析与可视化

### 5.1 基本结果解读

In [ ]:
# 结果分析示例
def analyze_results(results, config):
    """分析基准测试结果"""
    print("基准测试结果分析:")
    print("=" * 50)
    
    for framework in config["frameworks_to_test"]:
        print(f"\n{framework} 框架:")
        for n_qubits in config["n_qubits_range"]:
            if framework in results and n_qubits in results[framework]:
                data = results[framework][n_qubits]
                print(f"  {n_qubits} 量子比特:")
                print(f"    收敛率: {data['convergence_rate']:.1%}")
                if data['avg_time_to_solution'] is not None:
                    print(f"    求解时间: {data['avg_time_to_solution']:.3f} ± {data['std_time_to_solution']:.3f} 秒")
                print(f"    内存使用: {data['avg_peak_memory']:.1f} ± {data['std_peak_memory']:.1f} MB")
                if data['avg_final_error'] is not None:
                    print(f"    最终误差: {data['avg_final_error']:.2e}")
                print(f"    总评估次数: {data['avg_total_evals']:.1f} ± {data['std_total_evals']:.1f}")

# 分析结果
if single_results:
    analyze_results(single_results, single_framework_config)
elif multi_results:
    analyze_results(multi_results, multi_framework_config)
else:
    print("没有可分析的结果。")

### 5.2 可视化仪表盘

In [ ]:
# 生成标准仪表盘
def generate_dashboard(results, config, title_suffix=""):
    """生成可视化仪表盘"""
    if results:
        visualizer = VQEBenchmarkVisualizer(results, config)
        output_dir = config.get("system", {}).get("output_dir", "./benchmark_results_high_performance/")
        visualizer.plot_dashboard(output_dir)
        print(f"可视化仪表盘已生成{title_suffix}并保存到 {output_dir}")
    else:
        print(f"没有结果数据，无法生成仪表盘{title_suffix}。")

# 为不同结果生成仪表盘
if single_results:
    generate_dashboard(single_results, single_framework_config, " (单框架测试)")

if multi_results:
    generate_dashboard(multi_results, multi_framework_config, " (多框架比较)")

### 5.3 自定义可视化

#### 5.3.1 扩展性分析

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 自定义求解时间分析
def analyze_time_scaling(results, frameworks):
    """分析时间扩展性"""
    plt.figure(figsize=(10, 6))
    
    for fw in frameworks:
        qubits = []
        times = []
        errors = []
        for n_qubits in sorted(results[fw].keys()):
            data = results[fw][n_qubits]
            if data['avg_time_to_solution'] is not None:
                qubits.append(n_qubits)
                times.append(data['avg_time_to_solution'])
                errors.append(data['std_time_to_solution'])
        
        if qubits:  # 确保有数据点
            plt.errorbar(qubits, times, yerr=errors, marker='o', label=fw, capsize=5)
    
    plt.yscale('log')
    plt.xlabel('量子比特数')
    plt.ylabel('求解时间 (秒)')
    plt.title('框架扩展性比较：求解时间')
    plt.legend()
    plt.grid(True, which="both", ls="--")
    plt.show()

# 分析时间扩展性
if single_results:
    analyze_time_scaling(single_results, single_framework_config["frameworks_to_test"])
elif multi_results:
    analyze_time_scaling(multi_results, multi_framework_config["frameworks_to_test"])
else:
    print("没有结果数据可用于时间扩展性分析。")

#### 5.3.2 内存效率分析

In [ ]:
# 内存效率分析
def analyze_memory_efficiency(results, frameworks):
    """分析内存使用效率"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # 绝对内存使用
    for fw in frameworks:
        qubits = []
        memory = []
        for n_qubits in sorted(results[fw].keys()):
            data = results[fw][n_qubits]
            qubits.append(n_qubits)
            memory.append(data['avg_peak_memory'])
        
        if qubits:  # 确保有数据点
            ax1.plot(qubits, memory, 'o-', label=fw)
    
    ax1.set_xlabel('量子比特数')
    ax1.set_ylabel('峰值内存 (MB)')
    ax1.set_title('绝对内存使用')
    ax1.legend()
    ax1.grid(True)
    
    # 内存效率(每量子比特内存)
    for fw in frameworks:
        qubits = []
        efficiency = []
        for n_qubits in sorted(results[fw].keys()):
            data = results[fw][n_qubits]
            qubits.append(n_qubits)
            efficiency.append(data['avg_peak_memory'] / n_qubits)
        
        if qubits:  # 确保有数据点
            ax2.plot(qubits, efficiency, 's-', label=fw)
    
    ax2.set_xlabel('量子比特数')
    ax2.set_ylabel('每量子比特内存 (MB/qubit)')
    ax2.set_title('内存效率')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

# 分析内存效率
if single_results:
    analyze_memory_efficiency(single_results, single_framework_config["frameworks_to_test"])
elif multi_results:
    analyze_memory_efficiency(multi_results, multi_framework_config["frameworks_to_test"])
else:
    print("没有结果数据可用于内存效率分析。")

## 6. 高级应用与自定义

### 6.1 自定义问题定义

以下展示如何扩展框架以支持自定义问题类型。

In [ ]:
# 自定义问题基类
class CustomProblem:
    def __init__(self, problem_config):
        self.config = problem_config
    
    def build_hamiltonian(self, n_qubits, framework_wrapper):
        """构建特定于框架的哈密顿量"""
        raise NotImplementedError
    
    def get_exact_energy(self, n_qubits):
        """获取精确基态能量(如果可能)"""
        raise NotImplementedError

# 具体自定义问题示例：XY模型
class XYModel(CustomProblem):
    def build_hamiltonian(self, n_qubits, framework_wrapper):
        j_coupling = self.config.get("j_coupling", 1.0)
        gamma = self.config.get("gamma", 0.5)  # 各向异性参数
        
        # 这里需要根据具体框架实现
        # 这是一个示例，实际实现需要针对每个框架进行适配
        print(f"构建XY模型哈密顿量 (N={n_qubits}, J={j_coupling}, γ={gamma})")
        return None  # 实际实现应返回哈密顿量
    
    def get_exact_energy(self, n_qubits):
        # XY模型的解析解(如果可用)
        j_coupling = self.config.get("j_coupling", 1.0)
        gamma = self.config.get("gamma", 0.5)
        # 实现解析解或数值精确解
        print(f"计算XY模型精确能量 (N={n_qubits}, J={j_coupling}, γ={gamma})")
        return -n_qubits * j_coupling  # 简化示例

# 使用自定义问题
def run_custom_problem_benchmark():
    custom_problem_config = {
        "model_type": "XY_Model",
        "j_coupling": 1.0,
        "gamma": 0.5,
        "boundary_conditions": "periodic"
    }
    
    # 包装到配置中
    config = merge_configs({
        "problem": custom_problem_config,
        "n_qubits_range": [4, 6],  # 减少量子比特数以节省时间
        "frameworks_to_test": available_frameworks[:1] if len(available_frameworks) > 0 else ["Qiskit"],
        "n_runs": 2,  # 减少运行次数
        "experiment_name": "Custom_XY_Model_Benchmark"
    })
    
    print("自定义问题配置示例:")
    print(f"  模型类型: {custom_problem_config['model_type']}")
    print(f"  耦合强度: {custom_problem_config['j_coupling']}")
    print(f"  各向异性参数: {custom_problem_config['gamma']}")
    
    # 注意：实际运行需要扩展FrameworkWrapper以支持自定义问题
    print("\n注意：实际运行需要扩展FrameworkWrapper以支持自定义问题")
    return config

# 创建自定义问题配置
custom_config = run_custom_problem_benchmark()

### 6.2 自定义Ansatz结构

In [ ]:
# 自定义Ansatz基类
class CustomAnsatz:
    def __init__(self, ansatz_config):
        self.config = ansatz_config
    
    def build_circuit(self, n_qubits, framework_wrapper):
        """构建特定于框架的Ansatz电路"""
        raise NotImplementedError
    
    def get_parameter_count(self, n_qubits):
        """获取参数数量"""
        raise NotImplementedError

# 具体自定义Ansatz示例：硬件高效变体
class HardwareEfficientVariant(CustomAnsatz):
    def build_circuit(self, n_qubits, framework_wrapper):
        n_layers = self.config.get("n_layers", 2)
        rotation_style = self.config.get("rotation_style", "ry_rz")  # ry_rz, rx, ry, rz
        entanglement_pattern = self.config.get("entanglement_pattern", "linear")
        
        print(f"构建自定义Ansatz (N={n_qubits}, 层数={n_layers}, 旋转样式={rotation_style}, 纠缠模式={entanglement_pattern})")
        # 这里需要根据具体框架实现
        return None  # 实际实现应返回电路
    
    def get_parameter_count(self, n_qubits):
        n_layers = self.config.get("n_layers", 2)
        rotation_style = self.config.get("rotation_style", "ry_rz")
        
        if rotation_style == "ry_rz":
            return 2 * n_qubits * n_layers
        elif rotation_style in ["rx", "ry", "rz"]:
            return n_qubits * n_layers
        else:
            raise ValueError(f"不支持的旋转样式: {rotation_style}")

# 使用自定义Ansatz
def run_custom_ansatz_benchmark():
    custom_ansatz_config = {
        "ansatz_type": "HardwareEfficientVariant",
        "n_layers": 3,
        "rotation_style": "ry_rz",
        "entanglement_pattern": "circular"
    }
    
    config = merge_configs({
        "ansatz_details": custom_ansatz_config,
        "n_qubits_range": [4, 6],  # 减少量子比特数以节省时间
        "frameworks_to_test": available_frameworks[:1] if len(available_frameworks) > 0 else ["Qiskit"],
        "n_runs": 2,  # 减少运行次数
        "experiment_name": "Custom_Ansatz_Benchmark"
    })
    
    # 创建自定义Ansatz示例
    ansatz = HardwareEfficientVariant(custom_ansatz_config)
    param_count = ansatz.get_parameter_count(6)  # 示例：6量子比特
    
    print("自定义Ansatz配置示例:")
    print(f"  类型: {custom_ansatz_config['ansatz_type']}")
    print(f"  层数: {custom_ansatz_config['n_layers']}")
    print(f"  旋转样式: {custom_ansatz_config['rotation_style']}")
    print(f"  纠缠模式: {custom_ansatz_config['entanglement_pattern']}")
    print(f"  参数数量 (6量子比特): {param_count}")
    
    # 注意：实际运行需要扩展FrameworkWrapper以支持自定义Ansatz
    print("\n注意：实际运行需要扩展FrameworkWrapper以支持自定义Ansatz")
    return config

# 创建自定义Ansatz配置
custom_ansatz_config = run_custom_ansatz_benchmark()

## 7. 研究案例

### 7.1 案例1：不同规模问题的扩展性比较

**研究目标**：比较不同量子计算框架在问题规模增加时的性能扩展性

In [ ]:
# 扩展性比较实验
scalability_config = merge_configs({
    "n_qubits_range": [4, 6, 8, 10],  # 减少量子比特数以节省时间
    "frameworks_to_test": available_frameworks[:2] if len(available_frameworks) >= 2 else available_frameworks,
    "n_runs": 2,  # 减少运行次数以节省时间
    "experiment_name": "Framework_Scalability_Study",
    "ansatz_details": {
        "n_layers": 3,  # 减少层数以节省时间
        "entanglement_style": "linear"
    },
    "optimizer_details": {
        "max_evaluations": 300,  # 减少评估次数以节省时间
        "accuracy_threshold": 1e-4
    }
})

print("扩展性比较实验配置:")
print(f"  量子比特数范围: {scalability_config['n_qubits_range']}")
print(f"  测试框架: {scalability_config['frameworks_to_test']}")
print(f"  运行次数: {scalability_config['n_runs']}")
print(f"  Ansatz层数: {scalability_config['ansatz_details']['n_layers']}")
print(f"  最大评估次数: {scalability_config['optimizer_details']['max_evaluations']}")

In [ ]:
# 执行实验
print("\n开始扩展性比较实验...")
scalability_controller = BenchmarkController(scalability_config)
scalability_results = scalability_controller.run_all_benchmarks()
print("扩展性比较实验完成!")

#### 7.1.1 结果分析

In [ ]:
# 扩展性分析
def analyze_scalability(results, config):
    """分析框架扩展性"""
    frameworks = config["frameworks_to_test"]
    n_qubits_range = config["n_qubits_range"]
    
    # 创建多个子图
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle("框架扩展性分析", fontsize=16)
    
    # 1. 时间扩展性
    for fw in frameworks:
        qubits = []
        times = []
        for n_qubits in n_qubits_range:
            if fw in results and n_qubits in results[fw]:
                data = results[fw][n_qubits]
                if data['avg_time_to_solution'] is not None:
                    qubits.append(n_qubits)
                    times.append(data['avg_time_to_solution'])
        
        if qubits:  # 确保有数据点
            axes[0, 0].plot(qubits, times, 'o-', label=fw)
    
    axes[0, 0].set_xlabel('量子比特数')
    axes[0, 0].set_ylabel('求解时间 (秒)')
    axes[0, 0].set_title('时间扩展性')
    axes[0, 0].set_yscale('log')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    # 2. 内存扩展性
    for fw in frameworks:
        qubits = []
        memory = []
        for n_qubits in n_qubits_range:
            if fw in results and n_qubits in results[fw]:
                data = results[fw][n_qubits]
                qubits.append(n_qubits)
                memory.append(data['avg_peak_memory'])
        
        if qubits:  # 确保有数据点
            axes[0, 1].plot(qubits, memory, 's-', label=fw)
    
    axes[0, 1].set_xlabel('量子比特数')
    axes[0, 1].set_ylabel('峰值内存 (MB)')
    axes[0, 1].set_title('内存扩展性')
    axes[0, 1].set_yscale('log')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # 3. 收敛率变化
    for fw in frameworks:
        qubits = []
        convergence_rates = []
        for n_qubits in n_qubits_range:
            if fw in results and n_qubits in results[fw]:
                data = results[fw][n_qubits]
                qubits.append(n_qubits)
                convergence_rates.append(data['convergence_rate'])
        
        if qubits:  # 确保有数据点
            axes[1, 0].plot(qubits, convergence_rates, '^-', label=fw)
    
    axes[1, 0].set_xlabel('量子比特数')
    axes[1, 0].set_ylabel('收敛率')
    axes[1, 0].set_title('收敛率变化')
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    # 4. 精度变化
    for fw in frameworks:
        qubits = []
        errors = []
        for n_qubits in n_qubits_range:
            if fw in results and n_qubits in results[fw]:
                data = results[fw][n_qubits]
                if data['avg_final_error'] is not None:
                    qubits.append(n_qubits)
                    errors.append(data['avg_final_error'])
        
        if qubits:  # 确保有数据点
            axes[1, 1].plot(qubits, errors, 'd-', label=fw)
    
    axes[1, 1].set_xlabel('量子比特数')
    axes[1, 1].set_ylabel('最终误差')
    axes[1, 1].set_title('精度变化')
    axes[1, 1].set_yscale('log')
    axes[1, 1].legend()
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()

# 分析扩展性结果
analyze_scalability(scalability_results, scalability_config)

## 8. 最佳实践与建议

### 8.1 实验设计原则

In [ ]:
# 健壮的实验设置
def robust_experiment_setup():
    """确保实验的可重现性和可靠性"""
    # 确保可重现性
    config = merge_configs()
    config["system"]["seed"] = 42  # 固定随机种子
    
    # 资源限制设置
    config["system"]["max_memory_mb"] = 4096
    config["system"]["max_time_seconds"] = 1800
    
    # 验证配置
    from vqe_config import validate_config
    is_valid, errors = validate_config(config)
    if not is_valid:
        print("配置错误:", errors)
        return None
    
    return config

# 实验设计检查清单
def experiment_design_checklist(config):
    """实验设计检查清单"""
    print("实验设计检查清单:")
    print("=" * 40)
    
    checks = []
    
    # 1. 范围适当性
    n_qubits_range = config.get("n_qubits_range", [])
    if len(n_qubits_range) >= 3:
        checks.append("✓ 量子比特数范围适当")
    else:
        checks.append("✗ 量子比特数范围过窄")
    
    # 2. 统计可靠性
    n_runs = config.get("n_runs", 1)
    if n_runs >= 3:
        checks.append("✓ 运行次数足够统计可靠")
    else:
        checks.append("✗ 运行次数可能不足")
    
    # 3. 框架多样性
    frameworks = config.get("frameworks_to_test", [])
    if len(frameworks) >= 2:
        checks.append("✓ 包含多个框架进行比较")
    else:
        checks.append("✗ 仅包含单个框架")
    
    # 4. 资源限制合理性
    system_config = config.get("system", {})
    max_memory = system_config.get("max_memory_mb", 0)
    max_time = system_config.get("max_time_seconds", 0)
    
    if max_memory > 0 and max_time > 0:
        checks.append("✓ 设置了资源限制")
    else:
        checks.append("✗ 未设置资源限制")
    
    # 5. 结果保存
    save_results = system_config.get("save_results", False)
    if save_results:
        checks.append("✓ 配置了结果保存")
    else:
        checks.append("✗ 未配置结果保存")
    
    # 输出检查结果
    for check in checks:
        print(f"  {check}")
    
    # 计算通过率
    pass_rate = sum(1 for check in checks if check.startswith("✓")) / len(checks)
    print(f"\n实验设计质量: {pass_rate:.0%}")
    
    if pass_rate >= 0.8:
        print("实验设计良好，可以继续执行")
    elif pass_rate >= 0.6:
        print("实验设计基本可行，但建议改进")
    else:
        print("实验设计需要重大改进")

# 使用检查清单
config = robust_experiment_setup()
if config:
    experiment_design_checklist(config)
else:
    print("无法创建健壮的实验配置。")

## 总结

本指南介绍了如何使用 `vqe_bench_new.py` 进行量子计算框架的性能比较研究。主要内容包括：

1. **框架架构**：分层配置系统和核心组件设计
2. **配置系统**：从基础到高级的参数配置方法
3. **基准测试执行**：单框架和多框架的比较测试
4. **结果分析**：多种可视化方法和统计分析技术
5. **高级应用**：自定义问题、Ansatz和优化器的扩展方法
6. **研究案例**：扩展性、优化器性能和参数敏感性的实际应用
7. **最佳实践**：实验设计原则和结果解读注意事项

通过本指南，研究人员可以快速掌握该框架的使用，并根据自身研究需求进行定制和扩展，从而有效地进行量子计算框架的性能比较研究。